In [1]:
import pandas as pd
df = pd.read_csv("./keyterm_scored.csv")
del df['idx']
df.clauses_text_final = df.clauses_text_final.apply(eval)
df.voice = df.voice.apply(eval)
df.abstraction_score = df.abstraction_score.apply(eval)
df.reading_ease_normalized = df.reading_ease_normalized.apply(eval)
df.grading_level_normalized = df.grading_level_normalized.apply(eval) 
df.sgrank_normalized = df.sgrank_normalized.apply(eval)
df.sample(frac = 1).head(20)

,prompt,response,clauses_text_final,voice,abstraction_score,reading_ease_normalized,grading_level_normalized,sgrank_normalized
28,When I get mad,"unsanity, its a lovely swim",[unsanity],[Undefined],[0.14],[0.75],[0.75],[0.0]
61,When I get mad,"it takes a long time to build up, but I am slo...","[it takes a long time but, to build up, I am s...","[A_def, A_def, P_bevb_x, A_def]","[0.14, 0.12, 0.22]","[0.01, 0.01, 0.06]","[0.04, 0.02, 0.08]","[0.57, 0.0, 0.22]"
380,My main problem is,being in my head rather than fully present and...,[being in my head rather than fully present an...,"[A_pron_x, A_def, P_bevb_x, A_def, A_def, A_de...","[0.14, 0.14, 0.25, 0.22, 0.14, 0.14, 0.25, 0.2...","[0.13, 0.25, 0.2, 0.06, 0.17, 0.09, 0.06, 0.15...","[0.2, 0.26, 0.21, 0.08, 0.18, 0.1, 0.08, 0.21,...","[0.03, 0.0, 0.03, 0.0, 0.0, 0.03, 0.0, 0.06, 0..."
232,A good boss,A good boss is critical toward making a workpl...,"[A good boss is critical toward, making a work...","[P_bevb_x, A_def]","[0.25, 0.25]","[0.14, 0.27]","[0.17, 0.3]","[0.41, 0.09]"
263,When I am criticized,I may experience at first a little sting insid...,[I may experience at first a little sting insi...,"[P_bevb_x, A_def, P_bevb_x, P_bevb_x, P_get_x,...","[0.25, 0.25, 0.25, 0.14, 0.14, 0.22, 0.25, 0.2...","[0.16, 0.16, 0.25, 0.07, 0.1, 0.01, 0.08, 0.2,...","[0.21, 0.19, 0.29, 0.09, 0.12, 0.03, 0.12, 0.2...","[0.15, 0.04, 0.15, 0.0, 0.0, 0.0, 0.04, 0.17, ..."
176,Being with other people,Is being nested within the eternal field that ...,"[Is being nested within the eternal field, tha...","[P_bevb_x, A_def, A_def, A_def, P_bevb_x, P_be...","[0.22, 0.14, 0.25, 0.14, 0.14, 0.25, 0.14, 0.2...","[0.16, 0.14, 0.0, 0.21, 0.07, 0.06, 0.14, 0.11...","[0.19, 0.17, 0.0, 0.23, 0.09, 0.08, 0.17, 0.15...","[0.07, 0.0, 0.0, 0.07, 0.0, 0.04, 0.09, 0.07, ..."
138,Being with other people,Gives me so many ways to practice and evolve b...,"[Gives me so many ways, to practice and, evolv...","[A_def, A_def, A_def, P_bevb_x, P_bevb_x]","[0.14, 0.25, 0.12, 0.14]","[0.01, 0.09, 0.0, 0.1]","[0.03, 0.1, 0.01, 0.12]","[0.31, 0.0, 0.5, 0.0]"
387,Privacy,for me seems to be tied to shame. I used to b...,"[for me seems to be tied to shame, I used to b...","[P_bevb_x, P_bevb_x, P_bevb_x, P_bevb_x, P_bev...","[0.14, 0.22, 0.14, 0.22, 0.14, 0.25, 0.25, 0.2...","[0.02, 0.08, 0.05, 0.13, 0.05, 0.06, 0.06, 0.0...","[0.06, 0.12, 0.09, 0.15, 0.09, 0.08, 0.08, 0.0...","[0.1, 0.14, 0.1, 0.0, 0.05, 0.07, 0.27, 0.04, ..."
42,We could make the world a better place if,..if individuals and systems are given the spa...,[if individuals and systems are given the spac...,"[P_bevb_x, P_bevb_x, A_def, P_bevb_x, P_bevb_x...","[0.25, 0.14, 0.14, 0.22, 0.25, 0.14, 0.25, 0.2...","[0.21, 0.01, 0.16, 0.01, 0.34, 0.16, 0.16, 0.1...","[0.25, 0.02, 0.18, 0.04, 0.35, 0.19, 0.19, 0.1...","[0.02, 0.0, 0.1, 0.0, 0.0, 0.08, 0.02, 0.0, 0...."
236,I am,pretty content right now.,[pretty content right now],[Undefined],[0.25],[0.13],[0.15],[0.0]


In [2]:
df['idx_abstraction_score'] = df['abstraction_score'].apply(lambda arr: arr.index(max(arr)) if len(arr)>0 else -1)
df['idx_grading_level'] = df['grading_level_normalized'].apply(lambda arr: arr.index(max(arr)) if len(arr)>0 else -1)
df['idx_reading_ease'] = df['reading_ease_normalized'].apply(lambda arr: arr.index(max(arr)) if len(arr)>0 else -1)
df['idx_sgrank'] = df['sgrank_normalized'].apply(lambda arr: arr.index(max(arr)) if len(arr)>0 else -1)

"""
df['final_score'] = df.apply(lambda row : (row['final_abstraction_score'] + row['final_grading_level'] + row['final_reading_ease'] + row['final_sgrank'])/4, axis = 1)
df.final_score = df.final_score.apply(lambda x : round(x, 2))
opdf = df[['prompt', 'response', 'clauses_text_final', 'voice', 'final_abstraction_score', 'final_grading_level', 'final_reading_ease', 'final_sgrank', 'final_score']]
opdf.sample(frac = 1).head(10)
"""
df[['prompt', 'response', 'clauses_text_final', 'voice', 'idx_abstraction_score', 'idx_grading_level', 'idx_reading_ease', 'idx_sgrank']].sample(frac = 1).head()

,prompt,response,clauses_text_final,voice,idx_abstraction_score,idx_grading_level,idx_reading_ease,idx_sgrank
17,My conscience bothers me if,I hurt myself or others.,[I hurt myself or others],[A_def],0,0,0,0
35,When I am criticized,it is an utterance from a particular observer ...,[it is an utterance from a particular observer...,"[P_bevb_x, P_bevb_x, A_def, P_bevb_x, A_def, A...",0,0,2,0
359,My father,goes to work,"[goes, to work]","[A_def, A_def]",0,1,0,1
53,Being with other people,is like watching life itself unfold through th...,"[is like, watching life itself, unfold through...","[P_bevb_x, A_def, A_def]",1,1,1,2
311,The past,confusing,[confusing],[Undefined],0,0,0,0


In [3]:
df['voice_abstraction'] = df.apply(lambda row : row['voice'][row['idx_abstraction_score']] if row['idx_abstraction_score'] >=0 else "Indeterminate", axis = 1)
df['voice_grading_level'] = df.apply(lambda row : row['voice'][row['idx_grading_level']] if row['idx_grading_level'] >=0 else "Indeterminate", axis = 1)
df['voice_reading_ease'] = df.apply(lambda row : row['voice'][row['idx_reading_ease']] if row['idx_reading_ease'] >=0 else "Indeterminate", axis = 1)
df['voice_sgrank'] = df.apply(lambda row : row['voice'][row['idx_sgrank']] if row['idx_sgrank'] >=0 else "Indeterminate", axis = 1)

df['final_abstraction'] = df.apply(lambda row : row['abstraction_score'][row['idx_abstraction_score']] if row['idx_abstraction_score'] >=0 else -1, axis = 1)
df['final_grading_level'] = df.apply(lambda row : row['grading_level_normalized'][row['idx_grading_level']] if row['idx_grading_level'] >=0 else -1, axis = 1)
df['final_reading_ease'] = df.apply(lambda row : row['reading_ease_normalized'][row['idx_reading_ease']] if row['idx_reading_ease'] >=0 else -1, axis = 1)
df['final_sgrank'] = df.apply(lambda row : row['sgrank_normalized'][row['idx_sgrank']] if row['idx_sgrank'] >=0 else -1, axis = 1)

df[['prompt', 'response', 'clauses_text_final', 'voice', 'final_abstraction', 'voice_abstraction', 'final_grading_level', 'voice_grading_level', 'final_reading_ease', 'voice_reading_ease', 'final_sgrank', 'voice_sgrank']].head()
#df.iloc[342]

,prompt,response,clauses_text_final,voice,final_abstraction,voice_abstraction,final_grading_level,voice_grading_level,final_reading_ease,voice_reading_ease,final_sgrank,voice_sgrank
0,The past,"Winds through us, both from our lives and cult...",[Winds through us both from our lives and cult...,"[A_pron_x, A_pron_x, P_bevb_x, P_bevb_x, P_bev...",0.25,A_pron_x,0.42,A_pron_x,0.26,A_pron_x,0.12,P_bevb_x
1,At times I worry about,Insufficient care and attention for his egocen...,[Insufficient care and attention for his egoce...,[A_pron_x],0.25,A_pron_x,0.49,A_pron_x,0.31,A_pron_x,0.08,A_pron_x
2,What gets me into trouble is,not considering others possibilities.,[not considering others possibilities],[A_def],0.25,A_def,0.52,A_def,0.51,A_def,0.00,A_def
3,When people are helpless,They often don&#039;t know it so they flak aro...,"[They often don, t know it, so they flak aroun...","[P_bevb_x, A_def, A_def, A_def]",0.14,P_bevb_x,0.12,A_def,0.10,A_def,0.00,P_bevb_x
4,Privacy,is a sense of hiding from others that which yo...,"[is a sense of, hiding from others that, which...","[P_bevb_x, A_def, P_bevb_x, P_bevb_x, A_def, A...",0.25,P_bevb_x,0.21,A_def,0.20,A_def,0.65,P_bevb_x


In [5]:
df['sentinel'] = "Indeterminate"
cdict = {-1 : 'sentinel', 0 : 'voice_abstraction', 1 : 'voice_grading_level', 2 : 'voice_reading_ease', 3 : 'voice_sgrank'}
def get_final_voice(row):
    arr = [row['final_abstraction'], row['final_grading_level'], row['final_reading_ease'], row['final_sgrank']]
    idx = arr.index(max(arr))
    return row[cdict[idx]]
    
df['final_score'] = df.apply(lambda row: (row['final_abstraction'] + row['final_grading_level'] + row['final_reading_ease'] + row['final_sgrank'])/4 if row['final_abstraction'] >=0 else -1 , axis = 1)
df['final_voice'] = df.apply(get_final_voice, axis = 1)
df.final_score = df.final_score.apply(lambda x : round(x, 2))
df[['response', 'clauses_text_final', 'final_score', 'final_voice']].sample(frac = 1).head(20)

,response,clauses_text_final,final_score,final_voice
468,is best left in the past... Tomorrow presents ...,"[is, best left in the past Tomorrow, presents ...",0.35,A_def
207,good.,[good],0.19,Undefined
128,great!,[great],0.03,Undefined
253,the Lost World.,[the Lost World],0.04,A_def
469,there were no humans left on the planet... Whe...,"[there were no humans, left on the planet, Whe...",0.22,P_bevb_x
95,comes in many different forms.,[comes in many different forms],0.29,A_def
36,let it be a moment in time a need defined a st...,"[let, it be a moment in time a need, defined a...",0.30,A_def
252,I sought to find out 'why?' . Knowing the 'why...,"[I sought to find out, why Knowing, the why ca...",0.14,A_def
504,People who step out of line are interesting.,"[People, who step out of line, are interesting]",0.27,P_bevb_x
70,we need to give them the strength to help them...,"[we need to give them the strength, to help th...",0.20,A_def


In [6]:
df[['response', 'clauses_text_final', 'final_score', 'final_voice']].to_csv("./output.csv", index = False)
df[['prompt', 'response', 'clauses_text_final', 'voice', 'abstraction_score', 'reading_ease_normalized', \
       'grading_level_normalized', 'sgrank_normalized',   \
    'final_abstraction', 'final_grading_level', 'final_reading_ease', 'final_sgrank', 'final_score', \
       'voice_abstraction', 'voice_grading_level', 'voice_reading_ease', 'voice_sgrank', 'final_voice']].to_csv("./debug.csv", index = False)

print("Written")

Written
